dataset code: A301

eligibility:
1. after 2016
2. at least 2 admissions
3. admission is eligible if there is at least 1 recognized ccs code
5. episodes can't have multiple diagnostics assigned with more than 1 day delay between each other

process:
1. ccs codes
2. 12 months target window
3. saves delta_days and date of last admission before prediction_period of all datapoints

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [2]:
import ast
import pandas as pd
idx = pd.IndexSlice
pd.options.display.max_columns = None
pd.set_option("display.max_rows", None, "display.max_columns", None,'display.max_colwidth', None)

import numpy as np
import os

from datetime import timedelta
import json

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from ICDMappings import ICDMappings
icdmap = ICDMappings()

# Parameters

In [6]:
#where to save the resulting dataset
dataset_filepath = 'data/model_ready_dataset/icare2021_diag_A302/dataset.json'
raw_data_filepath = 'data/model_ready_dataset/icare2021_diag_A302/dataset.csv'

#checks

assert os.path.dirname(dataset_filepath) == os.path.dirname(raw_data_filepath), 'make sure both dataset are saved under the same directory'
assert os.path.isdir(os.path.dirname(dataset_filepath)),'Please create the directory first or try another path to save'

assert not os.path.isfile(dataset_filepath) or not os.path.isfile(raw_data_filepath), 'File exists, are you sure you want to overwrite it? If so, comment this line and run the notebook again'

----

In [3]:
class Icare:
    def __init__(self,data_folder):
        self.data_folder = data_folder
        self.diagnoses_path = 'LS_ANALYTICS.ICARE_CLINICO_DIAGNOSTICOS/index.csv'
        self.atividade_path = 'LS_ANALYTICS.ICARE_ATIVIDADE_HOSPITALAR/index.csv'
        
    
    def _read_diagnoses(self):
        print('Reading diagnostics table...')
        diagnoses_file = os.path.join(self.data_folder,self.diagnoses_path)
        
        df = pd.read_csv(diagnoses_file,sep='\t')
        
        print('begining: ',round(df.memory_usage(index=True).sum() / 1_000_000,1),'Mb') #Mbytes
        
        df = df.drop(columns='UNIDADE')
        #print('drop UNIDADE:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        df = df.drop(columns='DATA_FIM')
        #print('drop date_end:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        df.loc[:,'PRIORIDADE_DIAGNOSTICO'] = df.PRIORIDADE_DIAGNOSTICO.astype('category')
        #print('PRIORIDADE_DIAGNOSTICO to category:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        df = df.drop(columns=['ICD9_DESCRICAO'])
        #print('drop icd9_descricao:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        df.loc[:,'DIAGNOSTICO_PRINCIPAL'] = df.DIAGNOSTICO_PRINCIPAL.map({'S':True,'N':False})        
        #print('DIAGNOSTICO_PRINCIPAL to boolean:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        # a single row with the year 9064 ruins pd.to_datetime (overflow since pandas uses miliseconds in dates)
        # tldr:  we will remove that row
        nrows_before = df.shape[0]
        df = df.drop(df.DATA_INICIO.apply(lambda x: x[:4] if x is not np.nan else x).astype(float).where(lambda x: x==9064.0).dropna().index[0])
        nrows_after = df.shape[0]
        assert nrows_before == nrows_after + 1, 'Ooops, expecting to drop exactly 1 row. maybe dataset changed.'
        
        df.loc[:,'DATA_INICIO'] = pd.to_datetime(df.DATA_INICIO,format='%Y-%m-%d %H:%M:%S')
        #print('DATA_INICIO to datetime:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        df = df.drop_duplicates()
        #print('drop duplicates:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        # last row is trash
        df = df.iloc[:-1]
        
        print('end: ',round(df.memory_usage(index=True).sum() / 1_000_000),'Mb') #Mbytes
        print('Done')
        return df
        

In [4]:
icare = Icare('../../icare-dataset_2021-08')

In [5]:
df = icare._read_diagnoses()
df.head(1)

Reading diagnostics table...


/tmp/ipykernel_17945/365396413.py:12: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(diagnoses_file,sep='\t')


begining:  178.9 Mb
end:  117 Mb
Done


,EPISODIO,NHC,PRIORIDADE_DIAGNOSTICO,ICD9,DIAGNOSTICO_PRINCIPAL,DATA_INICIO
0,9B91F36DDC2E5D45,7E0717CE88F09A1EB9B7CB48F3520501,ASAP,2578,True,2016-10-19 17:40:00


In [10]:
res = df.groupby(['EPISODIO','NHC']).DATA_INICIO.agg([min,max])
res.head(3)

,,min,max
EPISODIO,NHC,,
000016F136985F31,9D73829E86FD907A3B2DBCB658D43107,2018-05-29 11:00:00,2018-05-29 11:00:00
00002A60B8C0E69D33E22BC4C4938ACB,E34C3A6E5A9DFDCC61EFFE9C3E7BF5D7,2020-10-19 23:47:00,2020-10-19 23:47:00
00003451D06E035DCC1AF5B2F032B4FA,8FFF5F2B2CFDE2CAE785CCD06B921A5E,2021-05-07 17:38:00,2021-05-07 17:38:00


----

# ICD9 to ICD9_3 then CCS

In [11]:
df.loc[:,'ICD9_3'] = df.ICD9.apply(lambda x: x[:3] if x is not np.nan else x)
df.loc[:,'ICD9_3->CCS'] = icdmap.lookup('icd9_3toccs',df['ICD9_3'])

## define eligibility criteria

1. data after 2016
1. patients with at least 2 admissions
1. all admissions must have at least 1 ccs diagnostic that is eligible

## define windows

3,6,12 months

what metrics to keep track of:
1. distribution of #admissions per target window
1. distribution of # diagnoses per target window
1. distribution of # admissions of input per target window
1. distribution # diagnoses of input per target window

In [12]:
# eligibility
df = icare._read_diagnoses()

print('Preparing eligibility filtering...')
df['ICD9_3'] = df.ICD9.apply(lambda x: x[:3] if x is not np.nan else x)
df['ICD9_3->CCS'] = icdmap.lookup('icd9_3toccs',df['ICD9_3'])

## after 2016
df = df.loc[df.DATA_INICIO > '2016-01-01']

## admission is eligible if there is at least 1 recognized ccs code
df['is_ccs_na'] = df['ICD9_3->CCS'].isna()
admissions_without_any_eligible_ccs = df.groupby('EPISODIO')['is_ccs_na'].all().where(lambda x: x == True).dropna().index
df = df.loc[~df.EPISODIO.isin(admissions_without_any_eligible_ccs)]

## drop rows where diagnostic is not recognized
df = df.loc[~df['ICD9_3->CCS'].isna()]

## patient with at least 2 admissions
patients_2_admissions = df.groupby('NHC').EPISODIO.size().where(lambda x: x > 1).dropna().index
df = df.loc[df.NHC.isin(patients_2_admissions)]

## episodes with multiple diagnostics can't have diagnostics assigned with more than 1 day delay between each other
episodes_far_diagnostics = df.groupby('EPISODIO').DATA_INICIO.agg([min,max]).diff(axis=1)['max'].dt.days.where(lambda x: x > 0).dropna().index
df = df.loc[~df.EPISODIO.isin(episodes_far_diagnostics)]
print('Done')

df = df.sort_values('DATA_INICIO')
df.DATA_INICIO = pd.to_datetime(df.DATA_INICIO.dt.date,format='%Y-%m-%d')

df.shape
df.EPISODIO.nunique()
df.NHC.nunique()
df.groupby('NHC').EPISODIO.size().value_counts().rename('Distribution of #episodes per eligible patient')

Reading diagnostics table...


/tmp/ipykernel_30796/365396413.py:12: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(diagnoses_file,sep='\t')


begining:  178.9 Mb
end:  117 Mb
Done
Preparing eligibility filtering...


/tmp/ipykernel_30796/2163803607.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_ccs_na'] = df['ICD9_3->CCS'].isna()


Done


(2052106, 9)

1861671

262811

2      66898
3      38716
4      27255
5      20019
6      15764
7      12487
8      10253
9       8504
10      7174
11      6120
12      5234
13      4588
14      3951
15      3464
16      3048
17      2737
18      2357
19      2131
20      1920
21      1770
22      1553
23      1455
24      1246
25      1103
26      1062
27       941
28       826
29       780
30       699
31       645
32       614
33       541
34       497
36       418
35       415
37       394
38       339
39       329
40       321
41       287
1        287
42       244
44       222
43       220
45       185
46       170
47       155
49       147
48       141
52       125
50       121
51       113
54       113
53        99
55        96
57        94
56        93
58        89
60        77
61        61
59        60
63        58
62        53
65        48
70        40
66        40
71        39
67        38
64        35
69        35
74        34
68        31
72        31
73        28
78        27
81        24
77        24

In [13]:
df.loc[:,'ccs_list'] = df['ICD9_3->CCS'].apply(lambda x: [x])
df_ = df.groupby(['NHC','DATA_INICIO'])[['ccs_list']].sum()

In [14]:
test = df_.copy()

#### 1000 rows
1. 1.68
2. 1.08

#### 10_000 rows
1. 14.4 (8.5x)
2. 8.49 (7.9x)

#### 100_000 rows
1. 189 (112x) (13x)
2. 85 (78x) (10x)

In [ ]:
%%time
#assert 1==2,'Prevent myself from running this cell
m = 12 #months
res = (test
 .groupby('NHC')
 .apply(lambda subdf: 
        subdf.assign(target = 
                     subdf.apply(lambda row: 
                                 subdf
                                 .loc[idx[:,
                                         row.name[1]+timedelta(days=1):row.name[1]+timedelta(days=30*m)
                                         ],
                                      'ccs_list'
                                     ]
                                 .sum(),
                                 axis=1
                                ),
                     history = 
                     subdf.apply(lambda row:
                                 subdf
                                 .loc[idx[:,
                                          :row.name[1]+timedelta(days=1)
                                         ],
                                      'ccs_list'
                                     ]
                                 .tolist(),
                                 axis=1
                                ),
                    )
       )
      )

# Add feature: delta days

In [ ]:
res['delta_days'] = res.reset_index().groupby('NHC')['DATA_INICIO'].diff().dt.days.fillna(0).values

In [ ]:
res.loc[:,'target2'] = res.target.apply(lambda x: x if type(x) == list else [])

# Print out some distributions

In [ ]:
print(f'Number of non-empty targets out of {res.shape[0]}: {res[res.target != 0].shape[0]}')
print(f'Distribution of target size')
pd.concat([res.loc[res.target != 0,'target'].apply(len).value_counts(normalize=False).iloc[:15],
           res.loc[res.target != 0,'target'].apply(len).value_counts(normalize=True).iloc[:15]
          ],axis=1)

In [ ]:
res.loc[res.target != 0].groupby('NHC').delta_days.agg('median').describe()[['25%','50%','75%']].rename('Quartiles of all patients of the median delta_days of each patient, on eligible datapoints')

In [ ]:
res2 = res.reset_index(1).rename(columns={'DATA_INICIO':'DATA'})

In [ ]:
res2.groupby('NHC').size().describe()[['25%','50%','75%']].rename('Quartiles #admissions eligible per patient')

In [ ]:
print(f'{res2.index.get_level_values(0).nunique()} patients eligible out of {test.index.get_level_values(0).nunique()}')

----

# Save

In [112]:
res2[res2.index.get_level_values(0).isin(res2.groupby('NHC').size().where(lambda x: x == 1).dropna().index)].head(10)

,DATA,ccs_list,target,history,delta_days,target2
NHC,,,,,,
000CC6B6978241BF7EC26026D203B9D7,2016-09-19,"[663.0, 251.0]",0,"[[663.0, 251.0]]",0.0,[]
000EFD57732A014F89170BEF38ABB731,2018-12-18,"[25.0, 50.0, 99.0]",0,"[[25.0, 50.0, 99.0]]",0.0,[]
0024809E1E3E338E55ED20B549AB1EE9,2019-02-06,"[120.0, 238.0]",0,"[[120.0, 238.0]]",0.0,[]
002E33C4113944B16851E7BCE219B94B,2020-04-23,"[670.0, 251.0]",0,"[[670.0, 251.0]]",0.0,[]
003A4D0FE843EBE9AA7C0E15442B8416,2018-02-22,"[122.0, 159.0]",0,"[[122.0, 159.0]]",0.0,[]
003DB1A25AE80675B379FF0C42029AE3,2017-08-21,"[134.0, 99.0]",0,"[[134.0, 99.0]]",0.0,[]
0046B15F18F1434BACC78EB51A3AD6EC,2017-02-17,"[117.0, 663.0]",0,"[[117.0, 663.0]]",0.0,[]
005C74FBD728D5563D3E8B6EE6F2C90D,2016-01-27,"[149.0, 149.0]",0,"[[149.0, 149.0]]",0.0,[]
005DA06D9C44632D05B39575AC632EE3,2020-08-05,"[258.0, 89.0]",0,"[[258.0, 89.0]]",0.0,[]


csv

In [125]:
res2.loc['0000676389D1EE60EB48AF5693F3F3DE']

,DATA,ccs_list,target,history,delta_days,target2
NHC,,,,,,
0000676389D1EE60EB48AF5693F3F3DE,2016-02-28,[670.0],[670.0],[[670.0]],0.0,[670.0]
0000676389D1EE60EB48AF5693F3F3DE,2016-08-08,[670.0],0,"[[670.0], [670.0]]",162.0,[]


In [132]:
res2.loc['50AE695937A38D390DDA2C4FC574D594']

DATA                2021-01-20
ccs_list        [660.0, 155.0]
target                       0
history       [[660.0, 155.0]]
delta_days                 0.0
target2                     []
Name: 50AE695937A38D390DDA2C4FC574D594, dtype: object

In [131]:
a = [res2.loc['50AE695937A38D390DDA2C4FC574D594'].history]
a
for e in a:
    e

[[[660.0, 155.0]]]

[[660.0, 155.0]]

In [92]:
res2 = res2.astype({'DATA':str}) # so that json can save this
res2.to_csv(raw_data_filepath,index=True)

and dict

In [133]:
# where it all begins
data = {}

patients = res2.index.unique()
for idx,p in tqdm(enumerate(patients)):
    
    p_data = res2.loc[p]
    
    history = p_data['history']
    targets = p_data['target2']
    delta_days = p_data['delta_days']
    date_last_history = p_data['DATA']
    
    history = history if type(history) != pd.Series else history.iloc[-1]
    targets = [targets] if type(targets) != pd.Series else targets.tolist()
    delta_days = [delta_days] if type(delta_days) != pd.Series else delta_days.tolist()
    date_last_history = [date_last_history] if type(date_last_history) != pd.Series else date_last_history.tolist()
    
    data[p] = { 'ccs': #only ccs for now
               {
                   'history':history,
                   'targets':targets,
                   'extra_features':
                   {
                       'delta_days': delta_days,
                       'date_last_history': date_last_history
                   }
               }
              }

0it [00:00, ?it/s]

In [134]:
with open(dataset_filepath, 'w') as fp:
    json.dump(data, fp)

----

# test

In [ ]:
with open(dataset_filepath, 'r') as fp:
    test_data = json.load(fp)

In [ ]:
test_data['0000676389D1EE60EB48AF5693F3F3DE']['ccs']

# fetching a dataset and adding targets without history

In [25]:
original_dataset = 'data/model_ready_dataset/icare2021_diag_A301/dataset.csv'

In [8]:
original_dataset = 'data/model_ready_dataset/icare2021_diag_A301/dataset.csv'
df = pd.read_csv(original_dataset)
df.head(5)

df.target2 = df.target2.apply(ast.literal_eval)
print('one down')
df.history = df.history.apply(ast.literal_eval)
print('two down')
df.ccs_list = df.ccs_list.apply(ast.literal_eval)
print('three down')

# flatten list to help remove diags from target
df.loc[:,'history_flattened'] = df.history.apply(lambda x: [item for sublist in x for item in sublist])

# remove diags in history from target
df.loc[:,'target_without_history'] = df[['history_flattened','target2']].apply(lambda row: [diag for diag in row.target2 if diag not in row.history_flattened],axis=1)

# compute some statistics about the reduction
df.loc[:,'target_reduction'] = df.target2.apply(len) - df.target_without_history.apply(len)

df.target2.apply(len).sum()

df.target_reduction.sum()

# What was the reduction in target labels in % with the original?

(df.target2.apply(len).sum() - df.target_reduction.sum()) / df.target2.apply(len).sum()

# we lost 40% of the original targets
print('old target disttribution')
df.target2.apply(len).value_counts()[:5]
print('new target disttribution')
df.target_without_history.apply(len).value_counts()[:5]

,NHC,DATA,ccs_list,target,history,delta_days,target2
0,0000676389D1EE60EB48AF5693F3F3DE,2016-02-28,[670.0],[670.0],[[670.0]],0.0,[670.0]
1,0000676389D1EE60EB48AF5693F3F3DE,2016-08-08,[670.0],0,"[[670.0], [670.0]]",162.0,[]
2,00007823E1ED7B1C8BEA6C5F10ED1294,2016-01-06,[670.0],[670.0],[[670.0]],0.0,[670.0]
3,00007823E1ED7B1C8BEA6C5F10ED1294,2016-06-15,[670.0],0,"[[670.0], [670.0]]",161.0,[]
4,00007823E1ED7B1C8BEA6C5F10ED1294,2018-11-14,[663.0],[670.0],"[[670.0], [670.0], [663.0]]",882.0,[670.0]


one down
two down
three down


6915667

4051904

0.41409787371196444

old target disttribution


0    414178
1    329099
2    236673
3    174556
4    133687
Name: target2, dtype: int64

new target disttribution


0    867822
1    353774
2    206872
3    125021
4     80195
Name: target_without_history, dtype: int64

In [9]:
df.DATA = pd.to_datetime(df.DATA)

In [ ]:
df = df.set_index('NHC').sort_index()

In [22]:
#assert 1 == 2
# confirm dates are increasing inside each patient
df.groupby('NHC').DATA.apply(lambda subdf: subdf.is_monotonic_increasing).all()

True

In [34]:
df = df.astype({'DATA':str}) # so that json can save this

In [35]:
# where it all begins
data = {}

patients = df.index.unique()
for idx,p in tqdm(enumerate(patients)):
    
    p_data = df.loc[p]
    
    history = p_data['history']
    targets = p_data['target2']
    new_targets = p_data['target_without_history']
    delta_days = p_data['delta_days']
    date_last_history = p_data['DATA']
    
    history = history if type(history) != pd.Series else history.iloc[-1]
    targets = [targets] if type(targets) != pd.Series else targets.tolist()
    new_targets = [new_targets] if type(new_targets) != pd.Series else new_targets.tolist()
    delta_days = [delta_days] if type(delta_days) != pd.Series else delta_days.tolist()
    date_last_history = [date_last_history] if type(date_last_history) != pd.Series else date_last_history.tolist()
    
    data[p] = { 'ccs': #only ccs for now
               {
                   'history':history,
                   'targets':targets,
                   'new_targets':new_targets,
                   'extra_features':
                   {
                       'delta_days': delta_days,
                       'date_last_history': date_last_history
                   }
               }
              }

262811it [01:12, 3620.66it/s]


Save it

In [36]:
with open(original_dataset[:original_dataset.find('.csv')]+'_added_new_targets.json', 'w') as fp:
    json.dump(data, fp)
print('Saved!')

Saved!


In [37]:
df.head(3)

,DATA,ccs_list,target,history,delta_days,target2,history_flattened,target_without_history,target_reduction
NHC,,,,,,,,,
0000676389D1EE60EB48AF5693F3F3DE,2016-02-28,[670.0],[670.0],[[670.0]],0.0,[670.0],[670.0],[],1
0000676389D1EE60EB48AF5693F3F3DE,2016-08-08,[670.0],0,"[[670.0], [670.0]]",162.0,[],"[670.0, 670.0]",[],0
00007823E1ED7B1C8BEA6C5F10ED1294,2016-01-06,[670.0],[670.0],[[670.0]],0.0,[670.0],[670.0],[],1


In [39]:
df.target2.sample(frac=1).head(5)

NHC
FA003D4E991803047131A4E3F407B01E                                                                                   []
201626F0887BDBC7DC48F9144D85BC70    [163.0, 164.0, 163.0, 164.0, 235.0, 164.0, 99.0, 99.0, 163.0, 164.0, 205.0, 99.0]
195DD24D63758226846A19E8849FE281                                                                 [229.0, 99.0, 229.0]
5D8499DB929F8B9AFFF700DD12967939                                                                                   []
FACF391EC7A06486BBA89B0672F1F0DC                                                                       [122.0, 126.0]
Name: target2, dtype: object